In [109]:
# Import libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Step 1: Create a soup object from the home page

In [8]:
url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/'
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [29]:
restaurants = []
for i in soup.find('table',{'class':'table'}).find_all('a'):
    res_dict = {'name':i.text,'href':i.attrs['href'],}
    restaurants.append(res_dict)

In [122]:
restaurants[0:5]

[{'name': 'A&W Restaurants', 'href': 'restaurants/1.html'},
 {'name': "Applebee's", 'href': 'restaurants/2.html'},
 {'name': "Arby's", 'href': 'restaurants/3.html'},
 {'name': 'Atlanta Bread Company', 'href': 'restaurants/4.html'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'href': 'restaurants/5.html'}]

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [95]:
food = []
for restaurant in restaurants:
    res_each = requests.get(url+restaurant['href'])
    soup_each = BeautifulSoup(res_each.content,'lxml')
    
    # based on the column names, create a food template.
    food_dict_temp = {}
    for i in soup_each.find('table').find('thead').find_all('th'):
        food_dict_temp[i.text] = []
    temp_keys = list(food_dict_temp.keys())
    
    # iterate each sub category and create dictonay and append to list
    for i in soup_each.find('tbody').find_all('tr'):
        start = 0
        food_instant = food_dict_temp.copy()
        for value in i.find_all('td'):
            food_instant[temp_keys[start]] = value.text
            start += 1
        food_instant['restaurant']=restaurant['name']
        food.append(food_instant)
    

In [108]:
food[0:3]

[{'Name': 'Original Bacon Double Cheeseburger',
  'Category': 'Burgers',
  'Calories': '760',
  'Fat': '45',
  'Carbs': '45',
  'restaurant': 'A&W Restaurants'},
 {'Name': 'Coney (Chili) Dog',
  'Category': 'Entrees',
  'Calories': '340',
  'Fat': '20',
  'Carbs': '26',
  'restaurant': 'A&W Restaurants'},
 {'Name': 'Chili Fries',
  'Category': 'French Fries',
  'Calories': '370',
  'Fat': '15',
  'Carbs': '49',
  'restaurant': 'A&W Restaurants'}]

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [112]:
food_df = pd.DataFrame(food)

In [117]:
food_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5131 entries, 0 to 5130
Data columns (total 6 columns):
Name          5131 non-null object
Category      5131 non-null object
Calories      5131 non-null object
Fat           5131 non-null object
Carbs         5131 non-null object
restaurant    5131 non-null object
dtypes: object(6)
memory usage: 240.6+ KB


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [121]:
food_df.to_csv('food dataframe.csv',index=False)